In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout,BatchNormalization,Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.downloader as api
from gensim.models import Word2Vec
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2


dataset = load_dataset("go_emotions", "raw")

df = pd.DataFrame(dataset['train'])
text=df['text']
emotions=['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
          'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
          'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
          'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
           'remorse', 'sadness', 'surprise', 'neutral']

y=df[emotions]

# Tokenize and preprocess the text data
max_sequence_length = 142  # Set your desired sequence length
tokenizer = Tokenizer(num_words=33084)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
X = pad_sequences(sequences, maxlen=max_sequence_length)



# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)




# Create the LSTM model
model = Sequential()
model.add(Embedding(input_dim=33084, output_dim=150, input_length = 142, embeddings_initializer='uniform', trainable = False))
model.add(Dropout(0.25))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 512, kernel_regularizer = l2(0.01), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(28, kernel_regularizer = l2(0.01),activation='softmax'))




In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')


Epoch 1/10
5281/5281 [==============================] - 264s 47ms/step - loss: 1409769.2500 - accuracy: 0.1074 - val_loss: 48943.4766 - val_accuracy: 0.2639
Epoch 2/10
 505/5281 [=>............................] - ETA: 3:24 - loss: 5639732.5000 - accuracy: 0.0963

KeyboardInterrupt: ignored

In [ ]:
new_data=['i am scared','i excited today']
new_sequences = tokenizer.texts_to_sequences(new_data)
new_X = pad_sequences(new_sequences, maxlen=max_sequence_length)

# Use the model to predict emotions for the new data
predictions = model.predict(new_X)


1/1 [==============================] - 0s 21ms/step


In [ ]:
print(predictions)

[[2.9926507e-02 2.2783564e-02 3.8609773e-02 7.0007198e-02 5.5835050e-02
  1.7597180e-02 2.6767066e-02 6.6160232e-02 1.5459476e-02 3.8647488e-02
  4.3564074e-02 2.2858782e-02 3.9562914e-03 2.4247058e-02 1.3641274e-02
  1.2069280e-02 5.8900015e-03 2.4359792e-02 7.1691666e-03 1.2269825e-02
  4.1200347e-02 3.7120578e-03 2.7482226e-02 4.4563301e-03 8.1217485e-03
  3.6904711e-02 2.0194978e-02 3.0610856e-01]
 [2.7067131e-01 4.3167580e-02 1.1664976e-03 3.2008139e-03 3.7144706e-02
  7.9618394e-03 2.0932597e-03 9.4965585e-03 1.1429896e-02 2.2597644e-03
  2.6079675e-03 7.5711112e-04 1.8313041e-04 6.9965333e-02 5.3484435e-04
  1.0572503e-01 3.5907750e-04 1.4555353e-01 1.8037200e-01 6.2453363e-04
  3.6640324e-02 3.6203624e-03 4.7149309e-03 2.2822192e-03 3.3625323e-04
  1.6625887e-03 5.2586016e-03 5.0209928e-02]]


In [ ]:
predicted_emotion_labels = [emotion for emotion, prob in zip(emotions, predictions[0])]
print(predicted_emotion_labels)


['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
